In [1]:
from datasets import load_dataset
from collections import defaultdict
ds = load_dataset('hotpotqa/hotpot_qa', 'fullwiki', trust_remote_code=True)

/home/junseokkim00/anaconda3/envs/rp2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ds_train_rm_distract
import random

k=100
indices = random.sample(range(len(ds['train'])), k)
test_ds = ds['train'].select(indices)


In [3]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from dotenv import find_dotenv, load_dotenv
import os

load_dotenv(find_dotenv())
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
llm = ChatGroq(model='llama3-8b-8192')

In [9]:
human_msg="""You are an assistant for extracting evidence. Choose the best context that helps you to solve the given question. You should only pick a single context. Also return your choice in a number that indicates the passage, which are numerical values.

Question: {question}

Context: {context}

Best context:
"""
total_cnt=0
cnt=0
pred_og=[]
labels_og=[]
for data in test_ds:
    question, answer, sentences = data['question'], data['answer'], data['context']['sentences']
    print(data['supporting_facts']['title'])
    print(f"{total_cnt+1}/{len(test_ds)} Question: {question}")
    context=""
    context_dict={}
    for idx, sent_ls in enumerate(sentences):
        context+=f"{idx}. "
        context+=" ".join(sent_ls)
        context_dict[idx] = " ".join(sent_ls)
        context+='\n\n'
    print(f"context:\n {context}")
    chat_prompt = ChatPromptTemplate.from_messages([
        ("user", human_msg)
    ])
    chain = chat_prompt | llm
    output = chain.invoke({
        'question': question,
        'context': context
    }).content
    msg = human_msg + output + 'Now, please provide only the number that you chose without any explanation or preambles. '
    answer_extraction = ChatPromptTemplate.from_messages([
        ("user", msg)
    ])
    chain_ans = answer_extraction | llm
    final_output = chain_ans.invoke({
        'question': question,
        'context': context
    })
    print(final_output.content, answer)
    raise

# print(f"exact_match: {cnt} / {total_cnt}")
    
    


['Gary Halvorson', 'Alejandro Amenábar']
1/100 Question: Gary Halvorson and Alejandro Amenábar, have which occupation in common?
context:
 0. The Sea Inside (Spanish: "Mar adentro" ) is a 2004 Spanish drama film written, produced, directed, and scored by Alejandro Amenábar, which won the Academy Award for Best Foreign Language Film.  It is based on the real-life story of Ramón Sampedro (played by Javier Bardem), who was left quadriplegic after a diving accident, and his 28-year campaign in support of euthanasia and the right to end his life.

1. Tesis ("Thesis") is a 1996 Spanish thriller film.  It is the feature debut of director Alejandro Amenábar, and was written by Amenabar and Mateo Gil.  The film was made while he was still studying at Universidad Complutense in Madrid The movie won seven 1996 Goya Awards, including the award for Best Film, Best Original Screenplay, and Best Director.  It stars Ana Torrent, Fele Martínez and Eduardo Noriega.

2. Agora (Spanish: "Ágora" ) is a 200

RuntimeError: No active exception to reraise

In [11]:
import re

output = re.findall('\d+', "20")
print(output)

['20']
